<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-中类方法、类实例方法、静态方法有何区别?" data-toc-modified-id="Python-中类方法、类实例方法、静态方法有何区别?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python 中类方法、类实例方法、静态方法有何区别?</a></span></li><li><span><a href="#Python-中如何动态获取和设置对象的属性?" data-toc-modified-id="Python-中如何动态获取和设置对象的属性?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python 中如何动态获取和设置对象的属性?</a></span></li><li><span><a href="#单例模式的应用场景" data-toc-modified-id="单例模式的应用场景-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>单例模式的应用场景</a></span></li><li><span><a href="#线程安全的单例模式" data-toc-modified-id="线程安全的单例模式-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>线程安全的单例模式</a></span></li><li><span><a href="#线程安全的单例装饰器" data-toc-modified-id="线程安全的单例装饰器-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>线程安全的单例装饰器</a></span></li></ul></div>

## Python 中类方法、类实例方法、静态方法有何区别?   
<br/>  
* `类方法`: 是类对象的方法，在定义时需要在上方使用`@classmethod`进行装饰，形参为`cls`，表示类对象，类对象和实例对象都可调用
* `类实例方法`: 是类实例化对象的方法，只有实例对象可以调用，形参为`self`，指代对象本身
* `静态方法`: 是一个任意函数，在其上方使用`@staticmethod`进行装饰，可以用对象直接调用，静态方法实际上跟该类没有太大关系

## Python 中如何动态获取和设置对象的属性?
<br/>  
* `hasattr(object, name)`: 判断一个对象里面是否有 name 属性或者 name 方法, 返回 bool 值, 有 name 属性(方法)返回 True, 否则返回 False
* `getattr(object, name[,default])`: 获取对象 object 的属性或者方法, 如果存在则打印出来, 如果不存在,打印默认值, 默认值可选 (如果返回的是对象的方法, 则打印结果是: 方法的内存地址, 如果需要运行这个方法, 可以在后
面添加括号`()`)
* `setattr(object,name,values)`: 给对象的属性赋值, 若属性不存在, 先创建再赋值

In [ ]:
#-*- coding:utf-8 -*-

class Parent(object):
    x = 101                          # 类变量

    def __init__(self):
        self.y = 202                 # 实例变量


if __name__ == '__main__':

    if hasattr(Parent, 'x'):
        print getattr(Parent, 'x')   # 101
        setattr(Parent, 'x', 1001)
    print getattr(Parent, 'x')       # 1001


    if hasattr(Parent, 'y'):
        print getattr(Parent, 'y')


    p = Parent()
    if hasattr(p, 'y'):
        print getattr(p, 'y')        # 202
        setattr(p, 'y', 2002)
    print getattr(p, 'y')            # 2002


    if hasattr(p, 'x'):
        print getattr(p, 'x')
        setattr(p, 'x', 11001)      # 1001
    print getattr(p, 'x')           # 11001


## 单例模式的应用场景
<br/>  
* 资源共享的情况下, 避免由于资源操作时导致的性能或损耗等。如日志文件, 应用配置
* 控制资源的情况下, 方便资源之间的互相通信。如线程池等。 1.网站的计数器 2.应用配置 3.多线程池 4.
数据库配置,数据库连接池 5.应用程序的日志应用

```python
class A(object):

    _instance = None

    def __new__(cls, *args, **kwargs):

        if cls._instance is None:
            cls._instance = super(A, cls).__new__(cls, *args, **kwargs)

        if not isinstance(cls._instance, A):
            cls._instance = super(A, cls).__new__(cls, *args, **kwargs)

        return cls._instance
```

## 线程安全的单例模式

```python
#-*- coding:utf-8 -*-

import threading

def synchronized(func):
    func.__lock__ = threading.Lock()

    def lock_func(*args, **kwargs):
        with func.__lock__:
            return func(*args, **kwargs)
    return lock_func


class Singleton(object):
    _instance = None

    @synchronized
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(Singleton, cls).__new__(cls, *args, **kwargs)
        return cls._instance


def worker():
    s = Singleton()
    print 'id =========> {}'.format(id(s))

if __name__ == "__main__":

    task_list = []
    for one in range(30):
        t = threading.Thread(target=worker)
        task_list.append(t)

    for one in task_list:
        one.start()

    for one in task_list:
        one.join()
```

## 线程安全的单例装饰器

```python
import threading

def synchronized(func):
    func.__lock__ = threading.Lock()

    def synced_func(*args, **kws):
        with func.__lock__:
            return func(*args, **kws)

    return synced_func

def Singleton(cls):
    instances = {}

    @synchronized
    def get_instance(*args, **kw):
        if cls not in instances:
            instances[cls] = cls(*args, **kw)
        return instances[cls]

    return get_instance


@Singleton
class test(object):
    pass

def worker():
    single_test = test()
    print "id----> %s" % id(single_test)

if __name__ == "__main__":
    task_list = []
    for one in range(30):
        t = threading.Thread(target=worker)
        task_list.append(t)

    for one in task_list:
        one.start()

    for one in task_list:
        one.join()
```

In [ ]:
#-*- coding:utf-8 -*-
import time
import functools


def track_exception(f=None, maxRetries=3, delay=1, step=0, ingoreException = False,
                   exceptions=None, validate=None, default=None):

    """
    函数执行出现异常时自动重试的简单装饰器
    :param f: function 执行的函数。
    :param maxRetries: int 最多重试次数。
    :param delay: int/float 每次重试的延迟，单位秒。
    :param step: int/float 每次重试后延迟递增，单位秒。
    :param exceptions: BaseException/tuple/list 终止重试的异常类型
        单个异常直接传入异常类型，多个异常以tuple或list传入。
    :param validate: 验证函数，用于验证执行结果，并确认是否继续重试。
        函数签名应接收一个参数，每次被装饰的函数完成且未抛出任何异常时，调用验证函数，将执行的结果传入。
        如验证函数正常执行，且返回False，则继续重试，即使被装饰的函数完成且未抛出任何异常。
        如验证函数正常执行，没有返回值或返回除False以外的结果，则终止重试，并将函数执行结果返回。
        如验证函数抛出异常，且异常属于被重试装饰器捕获的类型，则继续重试。
        如验证函数抛出异常，且异常不属于被重试装饰器捕获的类型，则将验证函数的异常抛出。
    :param default: 默认值/默认值生成函数
    :return: 被装饰函数的执行结果。
    """


    if exceptions is not None:
        if not isinstance(exceptions, list):
            exceptions = [exceptions]


    def decorator(func):
        @functools.wraps(func)

        def wrapper(*args, **kwargs):
            func_exec = None

            i = 0
            while i < int(maxRetries):
                print 'i = {}'.format(i)
                try:
                    result = func(*args, **kwargs)

                    # 调用 validate 方法进行结果检查 (validate 只接受一个参数)
                    if callable(validate) and validate(result) is False:
                        continue
                    else:
                        return result

                except Exception as ex:
                    func_exec = ex

                    # 如果指定了 exceptions 参数, 并且抛出的异常隶属于 exceptions
                    # 则立即终止程序(不再进行遍历), 并抛出异常
                    if exceptions is not None:
                        for exception in exceptions:
                            if isinstance(func_exec, exception):

                                # 如果 default 是一个函数, 则调用这个函数
                                if callable(default):
                                    return default()

                                # 如果 default 是一个字符串, 则直接返回这个字符串
                                if default is not None:
                                    return default

                                # 如果 default = None, 直接抛出异常
                                raise func_exec
                finally:
                    i += 1
                    time.sleep(delay + step * i)

            else:
                # 如果 default 是一个函数, 则调用这个函数
                if callable(default):
                    return default()

                # 如果 default 是一个字符串, 则直接返回这个字符串
                if default is not None:
                    return default

                # 如果忽略Exception, 则以字符串形式返回异常内容; 否则抛出异常
                if not ingoreException:
                    raise func_exec
                else:
                    return func_exec

        return wrapper

    if callable(f):
        return decorator(f)
    return decorator




def divide_zero():
    return "Cannot division by zero"


# @track_exception(callback=sys.stdout.write, exceptions=[ZeroDivisionError])
# @track_exception(callback=sys.stdout.write, exceptions=ZeroDivisionError)
# @track_exception(callback=sys.stdout.write, exceptions=Exception, default="Cannot division by zero")
# @track_exception(callback=sys.stdout.write, exceptions=Exception, default=divide_zero)
# @track_exception(ingoreException=True)
@track_exception(exceptions=ZeroDivisionError)
def divide(a, b):
    return a / b


print divide(5, 0)
print divide(4, 2)